# Fahrplan Datenbereinigung

1. CSV einlesen
2. erste Rows mit unwichtigen Infos löschen (anhand er Indices) + alle Rows mit "Sprechertext" in der 2. Spalte löschen
3. Datenbereinigung von % Zeichen (Code aus anderem Notebook kopieren) + Einzelfälle von Hand ergänzen
    * inklusive: (V.O.) und evtl. weitere
4. Regieanweisungen:
    *  ~~loop über alles zum Filtern: alle Rows mit Regieanweisungen rausfiltern (wenn "Regie" in 2. Spalte, dann rausfiltern)~~
    *  ~~loop über gefilterten Text: suchen nach Text in [], speichern in Array~~
    *  ~~loop über alles: suchen nach Text in Arrays, prüfen ob Regieanweisung vor oder nach dem Sprechertext steht~~
    *  ~~daraus eine eigene Zeile machen nach dem Muster Nr.|"Regie"|Text (wenn Regieanweisung vor dem Text, Zeile davor einfügen, wenn Regieanweisung nach dem Text, Zeile danach einfügen)~~
        * ~~Idee: Verschachtelte Liste. Jedes ausgelagerte Element bekommt eine Nr. mitgegeben, die um eins größer/kleiner ist als die der Herkunftszeile. Dann am Ende alle einfügen und anhand der Nr.-Zeile neu sortieren~~
    * NEU: Aufteilung von Aussagen und Regieanweisungen in zwei verschiedene CSVs
        * Kriterien für Regieanweisungs-CSV: a) "Regieanweisung" in Regie/Sprechertext-Spalte oder b) eckige Klammern in "Text"-Spalte
        * Nächster Schritt: In "Text"-Spalte alles außerhalb der Klammern sowie die Klammern selbst entfernen
5. Elbische Passagen aussortieren:
    * Suche nach Text in ()
    * überprüfen, ob es problematische Fälle gibt (Englisch - Elbisch - Übersetzung ODER: Elbisch - Übersetzung - Elbisch - Übersetzung) -> maßgeschneiderte Lösung für die Sätze, konkreten Wortlaut des Elbischen löschen
    * Regelfall: Alles vor der Übersetzung d.h. alles vor dem Text in runden Klammern löschen
6. Fortlaufende Nummer überschreiben: Row Indices verwenden


## Webscraping

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd

In [2]:
lotr_2 = requests.get('https://imsdb.com/scripts/Lord-of-the-Rings-The-Two-Towers.html')

scripts_2 = [lotr_2]

In [3]:
def get_scripts():
    # Schleife für jedes Script
    for movie, script in enumerate(scripts_2, 1):
        # webscrape text
        soup = BeautifulSoup(script.text, 'lxml')
        all_character_tags = soup.find('pre')
        
        # convert scraped bs4 element tags to string
        text = str(all_character_tags)
        
        # delete all tags
        text = re.sub(r'</?[a-zA-Z]+>', '', text)
        
        # split into text blocs 
        # Problem: mehrere Trenner - \r\n\r\n und \r\n \r\n)
        blocs = re.split(r'(\r\n\s?\r\n)', text)
        
        # delete empty blocs (does not work yet!!!!)
        # nice to have, but not necessarily needed
        # blocs = [i for i in blocs if i != '\r\n \r\n' or i != '\r\n\r\n']

        # print(blocs)
        
        # PRINT ALL BLOCS
        #for num, bloc in enumerate(blocs):
         #   print(num, bloc)
        
        # PRINT ONLY REGIEANWEISUNGEN
        #counter = 0
        #for num, bloc in enumerate(blocs):
            # matches Regieanweisungen, d.h. alle Blöcke, die direkt mit [ beginnen
            #if(re.match(r'^\s*\[', bloc)):
               # counter += 1
               # print(num, bloc)
        #print(f'Anzahl der Regieanweisungen: {counter}')
        # Ergenbis: 363 (ohne die Regieanweisungen, die direkt hinter dem Text stehen!)
        # Gesamtzahl an Regieanweisungen: 671 (Ergebnis der Suche nach [ in Skript 2)
        
        # PRINT ONLY SPRECHERTEXT
        #counter = 0
        #for num, bloc in enumerate(blocs):
           #  matches Sprechertext, d.h. alle Blöcke, die nicht mit [ beginnen
            #if(not re.match(r'^\s*\[', bloc)):
               # counter += 1
                # Sprechername ausgeben
                #if (re.match(r'^.*(?=(\r\n))', bloc)):
                   # name = re.match(r'^.*(?=(\r\n))', bloc).group(0)
                   # print(f'Name: {name}')
                # Sprechertext ausgeben
               # print(num, bloc)
                
       # print(f'Anzahl der Sprechertexte: {counter}')
        
        # WRITE TO CSV
        
        # Dateiname festlegen
        filename = f'lotr_skript2_dialog_regie_first_output.csv'

        # Datei löschen, falls sie existiert:
        if os.path.exists(f'data/{filename}'):
            os.remove(f'data/{filename}')
            
            
        with open(f'data/{filename}', 'a') as f:
            # Spaltenüberschriften
            f.write(f'Nr.|Sprecher/Regie|Text|Filmnr.\n')
            print(f'Nr.|Sprecher/Regie|Text|Filmnr.\n')
                
            # Blöcke durchgehen
            for num, bloc in enumerate(blocs):
                # Leerzeichen entfernen und Fragezeichen ersetzen (erzeugt beim Ausführen sonst Unicode-Error)
                bloc = bloc.replace("�", "%")   
                
                # match Regieanweisungen d.h. alle Blöcke, die direkt mit [ beginnen
                if(re.match(r'^\s*\[', bloc)):
                    # remove whitespace
                    regieText = bloc.strip()
                    regieText = re.sub(r'\r\n', '', regieText) # Zeilenumbrüche innerhalb des Texts
                    regieText = re.sub(r'\s+', ' ', regieText) # mehrfache Leerzeichen werden zu einem
                    
                    # write to CSV
                    # Bsp: 1|Regie|Frodo jumps.|                 
                    f.write(f'{num}|Regie|{regieText}|2\n')
                    print(f'{num}|Regie|{regieText}|2\n')
                    
                # match Sprechertext, d.h. alle Blöcke, die NICHT direkt mit [ beginnen
                elif(not re.match(r'^\s*\[', bloc)):
                    # überflüssiger whitespace vor dem Namen entfernen
                    # positives Lookahead, matched jegliche Art whitespace vor dem ersten Buchstaben
                    sprecherText = re.sub(r'^\s*(?=([a-zA-Z]))', "", bloc)
                    
                    # matched alles vor dem ersten Zeilenumbruch, d.h. den Sprechernamen
                    if (re.match(r'^.*(?=(\r\n))', sprecherText)):
                        # Name ohne Whitespace aus dem Text herausziehen
                        # group(0) wird benötigt, um auf den Text im Match-Objekt zuzugreifen, das von re.match zurückgegeben wird
                        name = re.match(r'^.*(?=(\r\n))', sprecherText).group(0).strip()
                        
                        # alles vor dem ersten Zeilenumbruch (inkl. Name wird gelöscht)
                        sprecherText = re.sub(r'^.*\r\n', '', sprecherText)
                        sprecherText = sprecherText.strip()
                        sprecherText = re.sub(r'\r\n', '', sprecherText) # Zeilenumbrüche innerhalb des Texts
                        sprecherText = re.sub(r'\s+', ' ', sprecherText) # mehrfache Leerzeichen werden zu einem
                        
                        # write to CSV
                        # only write if name is not empty (otherwise there will be empty rows in between)
                        if(name != ""):
                            f.write(f'{num}|{name}|{sprecherText}|2\n')
                            print(f'{num}|{name}|{sprecherText}|2\n')
                    else:
                        f.write(f'{num}|Sprechertext|{sprecherText}|2\n')
                        print(f'{num}|Sprechertext|{sprecherText}|2\n')
                
get_scripts()

Nr.|Sprecher/Regie|Text|Filmnr.

0|Sprechertext||2

2|Sprechertext|THE LORD OF THE RINGS: THE TWO TOWERS|2

4|Screenplay by|Peter Jackson, Fran Walsh and Philippa Boyens.|2

6|Based on "The Lord of The Rings" trilogy by|J.R.R Tolkien.|2

8|Sprechertext|Transcription credits|2

10|Accela, Aina, Bad burn, Bridget Chubb,|Brionn Equus (Lochrann), Drusilia, Elf Lady, %owyn Unquendor, Feanari, Finafyr, Flourish, Galadriel, Heri, Julamb, JustinsIce(Mdjasrie), Kazren, Krystal, Lady%owynKenobi, Lady Evenstar, Legolas%Bow, Lithorose, Melody, Mormegil, Nilmandra, Padfoot, Penwiper, Pilgrim Grey, Primula Baggins, Randy Savage, Samwise the Brave, Sirius Black, Tethra, The Lidless Eye, Turnar, Xyla, Yaksha|2

12|Elvish dialogue from The Elvish Linguistic|Fellowship.|2

14|Sprechertext||2

16|Regie|[TITLE: THE LORD OF THE RINGS]|2

18|Sprechertext||2

20|Regie|[Camera pans over the Misty Mountains as voices drift in from the background.]|2

22|GANDALF|You cannot pass!|2

24|FRODO|Gandalf!|2

26|GANDA

2080|GIMLI|[Protesting] What are you doing? Argh! What are you stopping for!|2

2082|Regie|[HALDIR HACKS AT A FEW URUK AS HE TURNS TOWARDS THE GATE. SUDDENLY, HE IS STABBED IN HIS ARM.]|2

2084|With a grimace, he kills the Uruk-hai|and looks down as his wound in seeming disbelief. An Uruk-hai comes up from behind him unnoticed and slices him on the neck. As Haldir goes down, he looks around him and sees his kin fallen among dead Uruk-hai.]|2

2086|ARAGORN|[Sees Haldir falling] HALDIR!! [He runs up the steps to Haldir%s side and catches him and he collapses. Haldir's head rolls back, his eyes empty and unseeing. Aragorn bows his head in grief. Then with a cry, he jumps onto a ladder and swings down to the ground, killing as he goes.]|2

2088|GAMLING|Brace the gate!|2

2090|SOLDIER|Hold them!|2

2092|TH%ODEN|[Drawing his sword] To the gate! Draw your swords!|2

2094|Regie|[Th%oden and his commanders come to the gate, which is under heavy attack. The Uruk-hai are knocking down the gate an

## Datenbereinigung

### 1. CSV laden

In [4]:
lotr = pd.read_csv('data/lotr_skript2_dialog_regie_first_output.csv', sep='|')
lotr

,Nr.,Sprecher/Regie,Text,Filmnr.
0,0,Sprechertext,NaN,2
1,2,Sprechertext,THE LORD OF THE RINGS: THE TWO TOWERS,2
2,4,Screenplay by,"Peter Jackson, Fran Walsh and Philippa Boyens.",2
3,6,"Based on ""The Lord of The Rings"" trilogy by",J.R.R Tolkien.,2
4,8,Sprechertext,Transcription credits,2
...,...,...,...,...
1213,2442,SM%AGOL,"Come on, hobbits. Long ways to go yet. Sm%agol...",2
1214,2444,Regie,"[He turns to walk on, with Frodo and Sam follo...",2
1215,2446,GOLLUM,Follow me.,2
1216,2448,Regie,[ Camera pans up over the forest and Ephel D%a...,2


### 2. Erste Rows mit unwichtigen Informationen + mit Attribut "Sprechertext" löschen

In [46]:
lotr = lotr[lotr["Nr."] > 20]
lotr = lotr[lotr["Sprecher/Regie"] != "Sprechertext"]
lotr

,Nr.,Sprecher/Regie,Text,Filmnr.
11,22,GANDALF,You cannot pass!,2
12,24,FRODO,Gandalf!,2
13,26,GANDALF,"I am a servant of the Secret Fire, wielder of ...",2
14,28,Regie,[Camera pans closer to the mountain side.],2
15,30,GANDALF,Argh! Go back to the shadow. The Dark Fire wil...,2
...,...,...,...,...
1212,2440,GOLLUM,Shh% [He pops out from hiding in front of the ...,2
1213,2442,SM%AGOL,"Come on, hobbits. Long ways to go yet. Sm%agol...",2
1214,2444,Regie,"[He turns to walk on, with Frodo and Sam follo...",2
1215,2446,GOLLUM,Follow me.,2


### 3. Fehlerhaftes Encoding bereinigen (%-Zeichen ersetzen)

In [48]:
for index, row in lotr.iterrows():
    
    # Clear character names in character column
    lotr.at[index, "Sprecher/Regie"] = re.sub("SM%AGOL", "SMEAGOL", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("GR%MA", "GRIMA", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("H%MA", "HAMA", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("NAZG%L", "NAZGUL", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("%OWYN", "EOWYN", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("%OMER", "EOMER", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("TH%ODEN", "THEODEN", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("TH%ODRED", "THEODRED", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("%OTHAIN", "EOTHAIN", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("UGL%K", "UGLUK", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("MA%HUR", "MAUHUR", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("GRISHN%KH", "GRISHNAKH", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("\(V.O.\)", "", lotr.at[index, "Sprecher/Regie"])
    lotr.at[index, "Sprecher/Regie"] = re.sub("With a grimace, he kills the Uruk-hai", "Regie", lotr.at[index, "Sprecher/Regie"])
    
    # Clear place names and person names
    
    lotr.at[index, "Text"] = re.sub("Barad-d%r", "Barad-dur", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Palant%r", "Palantir", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Khazad-d%m", "Khazad-dum", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Helm%s\s*Deep", "Helm's Deep", lotr.at[index, "Text"])  # covers line breaks
    lotr.at[index, "Text"] = re.sub("L%rien", "Lorien", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("M%ria", "Moria", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Lothl%rien", "Lothlorien", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("D%rin%s Tower", "Durin's Tower", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Ud%n", "Udun", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Sm%agol", "Smeagol", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Sm%a%gol", "Smeagol", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Gr%ma", "Grima", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("H%ma", "Hama", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Ham%", "Hama", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Nazg%l", "Nazgul", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("%owyn", "Eowyn", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("%omer", "Eomer", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("%omund", "Eomund", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Th%oden", "Theoden", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Th%odred", "Theodred", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("%othain", "Eothain", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Ugl%k", "Ugluk", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Ma%hur", "Mauhur", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Grishn%kh", "Grishnak", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Gl%in", "Gloin", lotr.at[index, "Text"])


    # replace apostrophe's
    
    ## I'm, you're etc.

    lotr.at[index, "Text"] = re.sub("I%m", "I'm", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("You%re", "You're", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("you%re", "you're", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("He%s", "He's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("he%s", "he's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("She%s", "She's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("she%s", "she's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("It%s", "It's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("it%s", "it's", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("We%re", "We're", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("we%re", "we're", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("They%re", "They're", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("they%re", "they're", lotr.at[index, "Text"])

    ## have/has, haven't/hasn't

    lotr.at[index, "Text"] = re.sub("i%ve", "i've", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("I%ve", "I've", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("you%ve", "you've", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("You%ve", "You've", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("we%ve", "we've", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("We%ve", "We've", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("they%ve", "they've", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("They%ve", "They've", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("haven%t", "haven't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Haven%t", "Haven't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("hasn%t", "hasn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Hasn%t", "Hasn't", lotr.at[index, "Text"])

    ## is, can, do

    lotr.at[index, "Text"] = re.sub("mustn%t", "mustn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Mustn%t", "Mustn't", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("let%s", "let's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Let%s", "Let's", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("can%", "can't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Can%t", "Can't", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("that%s", "that's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("That%s", "That's", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("there%s", "there's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("There%s", "There's", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("what%s", "what's", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("What%s", "What's", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("isn%t", "isn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Isn%t", "Isn't", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("aren%t", "aren't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Aren%t", "Aren't", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("don%t", "don't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Don%t", "Don't", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("doesn%t", "doesn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("doesn%t", "doesn't", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("didn%t", "didn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Didn%t", "Didn't", lotr.at[index, "Text"])

    ## was/will, wasn't/won't

    lotr.at[index, "Text"] = re.sub("I%ll", "I'll", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("you%ll", "you'll", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("You%ll", "You'll", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("he%ll", "he'll", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("He%ll", "He'll", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("she%ll", "she'll", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("She%ll", "She'll", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("it%ll", "it'll", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("It%ll", "It'll", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("we%ll", "we'll", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("We%ll", "We'll", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("they%ll", "they'll", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("They%ll", "They'll", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("wasn%t", "wasn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Wasn%t", "Wasn't", lotr.at[index, "Text"])

    lotr.at[index, "Text"] = re.sub("won%t", "won't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Won%t", "Won't", lotr.at[index, "Text"])

    ## Conjunctive (wouldn't, couldn't, should)

    lotr.at[index, "Text"] = re.sub("couldn%t", "couldn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("shouldn%t", "shouldn't", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("I%d", "I'd", lotr.at[index, "Text"])


    # Single cases

    lotr.at[index, "Text"] = re.sub("YOU% SHALL NOT... PASS!!!", "YOU SHALL NOT... PASS!!!", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("lovely % Lembas bread", "lovely Lembas bread", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("S... S...Smeagol%", "S... S...Smeagol.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Grima% Grima%", "Grima. Grima.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Ooh% They look tasty!", "Ooh! They look tasty!", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("small % only children", "small, only children", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Bur%rum", "Burarum", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("then% a tunnel", "then...a tunnel", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Oh, he% he must have died", "Oh, he ... he must have died", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Why should I% welcome you, Gandalf%", "Why should I welcome you, Gandalf", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Eowyn% Eowyn", "Eowyn Eowyn", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Simbelmyn%", "Simbelmyn%", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Because% because", "Because ... because", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Mur%derer%!", "Murderer!", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("%em", "them", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("of course ridiculous%", "of course ridiculous.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("my heart%", "my heart.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Sleep%", "Sleep.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Arwen%", "Arwen ...", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Brego% ", "Brego. ", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("And%ril", "Anduril", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Myyy% PRECIOUSSS", "Myyy PRECIOUSSS", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("but% I'm sorry", "but ... I'm sorry", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Where was Gon%%", "Where was Gon--", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("lord %", "lord.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("To whatever end%", "To whatever end.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("agreed % you", "agreed ... you", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("War, yes%", "War, yes.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("ooold%", "ooold", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("anything%", "anything", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("looong%", "looong", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("NAZG'L", "NAZGUL", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("they % Oh!!", "they ... Oh!!", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Hold on, Mr. Frodo%", "Hold on, Mr. Frodo.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("% Samwise the Brave", ": Samwise the Brave", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("they're dead%", "they're dead.", lotr.at[index, "Text"])
    lotr.at[index, "Text"] = re.sub("Shh%", "Shh.", lotr.at[index, "Text"])
    
    
    # Other question marks are either apostrophes or in Elvis/Entish/Old English and will be deleted later anyway
    
    lotr.at[index, "Text"] = re.sub("%", "'", lotr.at[index, "Text"])


### 4. Trennen von Aussagen und Regieanweisungen in zwei Datensätze + Index/Dialog-Nr. reseten

#### 4.1 Regieanweisungen

In [100]:
# filter stage directions
stage_directions = lotr[(lotr["Text"].str.contains("\[")) | (lotr["Sprecher/Regie"] == "Regie")]

# remove text outside brackets

for index, row in stage_directions.iterrows():
    # remove curved brackets in advance (data contain nested brackets like [...(..)..], makes things very hard)
    stage_directions.at[index, "Text"] = re.sub("[()]", "", stage_directions.at[index, "Text"])
    # remove content outside square brackets, regex expression idea from 41686d6564's answer at https://stackoverflow.com/questions/64740437/remove-text-outside-of-bracket
    stage_directions.at[index, "Text"] = re.sub("[^[\]]+(?=[[(]|$)", "", stage_directions.at[index, "Text"])
    # remove square brackets
    stage_directions.at[index, "Text"] = re.sub("[\[\]]", "", stage_directions.at[index, "Text"])
    # strip to remove whitespace in beginning and end
    stage_directions.at[index, "Text"] = stage_directions.at[index, "Text"].strip()


# print everything for control
for index, row in stage_directions.iterrows():
    print(index, row["Text"])
    
# reset index and Nr.-column
count = 1
for index, row in stage_directions.iterrows():
    stage_directions.at[index, "Nr."] = count
    count += 1
stage_directions.reset_index(drop=True, inplace=True)



14 Camera pans closer to the mountain side.
15 Camera zooms in through the mountain and focuses on Gandalf and the Balrog on the bridge of Khazad-dum. The Balrog strikes down on Gandalf with its flaming sword. Gandalf parries the blow with Glamdring, shattering the Balrog's sword.
16 Gandalf strikes his staff onto the bridge. As the Balrog steps forward, the bridge collapses from under it and the demon plunges backward into the chasm. Gandalf, exhausted, leans on his staff and watches the Balrog fall then turns to follow the others. At the last minute, the flaming whip lashes up from the depths of the abyss and winds around Gandalf's ankle, dragging him over the edge. He clings onto the bridge but is straining to keep his grip.
18 Frodo rushes forward but Boromir restrains him.
22 Gandalf loses his grip and falls into the chasm
24 Gandalf loses his grip and falls into the chasm
25 Calls after Gandalf as he falls into the abyss
26 Gandalf falls after the Balrog and grabs hold of his swo

#### 4.2 Speech

In [102]:
# filter lines that contain speech
speech = lotr[(lotr["Sprecher/Regie"] != "Regie")]

# remove text INSIDE square brackets (these are stage directions and unwanted)

for index, row in speech.iterrows():
    # remove text inside square brackets AND ALSO' brackets themselves
    speech.at[index, "Text"] = re.sub("\[[^\]]*\]", "", speech.at[index, "Text"])
    # strip to remove whitespace in beginning and end
    speech.at[index, "Text"] = speech.at[index, "Text"].strip()

# print lines for control
for index, row in speech.iterrows():
    #if "[" in row["Text"]:
        print(index, row["Text"])

# reset index and Nr.-column
count = 1
for index, row in speech.iterrows():
    speech.at[index, "Nr."] = count
    count += 1
speech.reset_index(drop=True, inplace=True)

11 You cannot pass!
12 Gandalf!
13 I am a servant of the Secret Fire, wielder of the Flame of Anor!
15 Argh! Go back to the shadow. The Dark Fire will not avail you, Flame of Udun!  YOU SHALL NOT... PASS!!!
17 Argh!
19 No! No!
20 Gandalf!
21 Fly, you fools!
23 Noooooooooooooooo!!!!
25 Gaaandaaaaalf!!
27 Gandalf!
28 What is it, Mr. Frodo?
29 Nothing. Just a dream.
31 Mordor. The one place in Middle-earth we don't want to see any closer, and the one place we're trying to get to. It's just where we can'tt get. Let's face it, Mr. Frodo, we're lost. I don't think Gandalf meant for us to come this way.
32 He didn't mean for a lot of things to happen, Sam... but they did.
34 Mr. Frodo? It's the Ring, isn't it?
35 It's getting heavier.
36 What food have we got left?
37 Well, let me see.  Oh yes, lovely Lembas bread. And look!  More lembas bread.
39 I don't usually hold with foreign food, but this Elvish stuff, it's not bad.
40 Nothing ever dampens your spirits, does it Sam?
42 Those rain cloud

### 5. Elbisches Original entfernen

In [92]:
# remove elvish (and westron) original lines and keep translation  

for index, row in speech.iterrows():
    # condition: must contain bracket
    if "(" in row["Text"]:
        # remove content outside brackets, regex expression idea from 41686d6564's answer at https://stackoverflow.com/questions/64740437/remove-text-outside-of-bracket
        speech.at[index, "Text"] = re.sub("[^[()]+(?=[(]|$)", "", speech.at[index, "Text"])
        # remove brackets
        speech.at[index, "Text"] = re.sub("[()]", "", speech.at[index, "Text"])
        # strip to remove whitespace in beginning and end
        speech.at[index, "Text"] = speech.at[index, "Text"].strip()

# print for control    
for index, row in speech.iterrows():
    print(index, row["Text"])

11 You cannot pass!
12 Gandalf!
13 I am a servant of the Secret Fire, wielder of the Flame of Anor!
15 Argh! Go back to the shadow. The Dark Fire will not avail you, Flame of Udun!  YOU SHALL NOT... PASS!!!
17 Argh!
19 No! No!
20 Gandalf!
21 Fly, you fools!
23 Noooooooooooooooo!!!!
25 Gaaandaaaaalf!!
27 Gandalf!
28 What is it, Mr. Frodo?
29 Nothing. Just a dream.
31 Mordor. The one place in Middle-earth we don't want to see any closer, and the one place we're trying to get to. It's just where we can'tt get. Let's face it, Mr. Frodo, we're lost. I don't think Gandalf meant for us to come this way.
32 He didn't mean for a lot of things to happen, Sam... but they did.
34 Mr. Frodo? It's the Ring, isn't it?
35 It's getting heavier.
36 What food have we got left?
37 Well, let me see.  Oh yes, lovely Lembas bread. And look!  More lembas bread.
39 I don't usually hold with foreign food, but this Elvish stuff, it's not bad.
40 Nothing ever dampens your spirits, does it Sam?
42 Those rain cloud

### 6. Index und Dialog-Nr. reseten

,Nr.,Sprecher/Regie,Text,Filmnr.
0,1,Regie,Camera pans closer to the mountain side.,2
1,2,GANDALF,Camera zooms in through the mountain and focus...,2
2,3,Regie,Gandalf strikes his staff onto the bridge. As ...,2
3,4,Regie,Frodo rushes forward but Boromir restrains him.,2
4,5,Regie,Gandalf loses his grip and falls into the chasm,2
...,...,...,...,...
623,624,Regie,He looks in the direction of the hobbits,2
624,625,GOLLUM,Softly and sinisterly,2
625,626,GOLLUM,He pops out from hiding in front of the hobbits.,2
626,627,Regie,"He turns to walk on, with Frodo and Sam follow...",2


## Ab hier: Unwichtig und alte Ideen, wird erstmal beibehalten

In [ ]:
#for index, row in lotr.iterrows():
 #   lotr.at[index, " Text "] = re.sub("Haldir", "Hubert", lotr.at[index, " Text "])

stage_directions_outsourced = []

# das hier funktioniert, d.h. man kann auf diese Weise filtern:
#temporary_lotr[temporary_lotr[' Text '].str.match(' Well')== True]

#for index, row in temporary_lotr.iterrows():
    # das funktioniert nicht
    #if (row[" Text "].str.match(' Well')== True):
    #    print(row[" Text "])
    # das auch nicht
    #if (temporary_lotr.at[index, " Text "].str.match(' Well')== True):
     #   print(row[" Text "])
    # funktioniert eingeschränkt: Matcht nur, wenn das pattern am Beginn des Strings steht
    #if (re.match(" Yes", row[" Text "])):
        #print(row[" Text "])

# [Regieanweisung]...Text...[Regieanweisung]        
temporary_lotr = lotr[(lotr["Text"].str.contains("^\[.*\].*\[.*\]$")) & (lotr["Sprecher/Regie"] != "Regie")]
print(len(temporary_lotr))

for index, row in temporary_lotr.iterrows():
    new_line = []
    bracket = re.match(r'^\[.*?\]', row['Text']).group(0)
    new_line.append(row["Nr."])
    new_line.append(row["Sprecher/Regie"])
    new_line.append(bracket)
    new_line.append(row["Filmnr."])
    print(bracket)
    stage_directions_outsourced.append(new_line)
    
print(stage_directions_outsourced)
for index, row in temporary_lotr.iterrows():
    print(row["Text"])